In [53]:
# https://stackoverflow.com/questions/43599018/is-there-a-way-to-get-matplotlib-path-contains-points-to-be-inclusive-of-boundar
#I do quite like this command in Jupiter notebook:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
#It makes things wider and not waste the space on your screen
import pandas as pd
import importlib
import csv,os
import numpy as np
import pandas as pd
import itertools
import pyproj
import shapely
import copy
from uafgi import gicollections

import uafgi.data.w21
importlib.reload(uafgi.data.w21)

<module 'uafgi.data.w21' from '/Users/eafischer2/harn/pismip6/uafgi/uafgi/data/w21.py'>

In [22]:
df = pd.read_pickle('outputs/stability/stability.df')    # many rows
select = pd.read_pickle('outputs/stability/03_select.df')   # Just 23 rows
df = pd.merge(df, select, how='left', on='ns642_key', suffixes=(None,'_DELETEME'))
drops = [x for x in df.columns if x.endswith('_DELETEME')]
df = df.drop(drops, axis=1)
print(len(df))

3864


In [58]:
df.columns

Index(['success', 'error_msg', 'ns481_grid', 'velocity_file', 'year',
       'sigma_max', 'w21_popular_name', 'w21_greenlandic_name', 'w21_coast',
       'w21_category', 'w21_Qr', 'w21_Qf', 'w21_Qm', 'w21_Qs',
       'w21_Qc_inferred', 'w21_qm', 'w21_qf', 'w21_qc', 'w21_mean_depth',
       'w21_min_depth', 'w21_quality_str', 'w21_area_grounded_1992_2017',
       'w21_area_grounded_1992_1997', 'w21_area_grounded_1998_2007',
       'w21_area_grounded_2008_2017', 'w21_mean_fjord_width',
       'w21_length_grounded_1992_2017', 'w21_length_grounded_1992_1997',
       'w21_length_grounded_1998_2007', 'w21_length_grounded_2008_2017',
       'w21_ocean_model_sample_area', 'w21_mean_TF_1992-2017',
       'w21_mean_TF_1992-1997', 'w21_mean_TF_1998-2007',
       'w21_mean_TF_2008-2017', 'w21_subglacial_discharge_1992_2017',
       'w21_subglacial_discharge_1992_1997',
       'w21_subglacial_discharge_1998_2007',
       'w21_subglacial_discharge_2008_2017', 'w21_mean_xsection_area',
       'w21_me

In [109]:
select[['w21_popular_name', 'w21_category', 'w21_mean_undercutting_2008_2017', 'ns481_key', 'retreat_rate']]\
  .sort_values(['w21_category','w21_mean_undercutting_2008_2017'])


,w21_popular_name,w21_category,w21_mean_undercutting_2008_2017,ns481_key,retreat_rate
6,Ussing Br.,CR,0.88,W73.75N,-26.064922
18,Skinfaxe Gl.,CR,0.94,E63.35N,0.242703
22,Upernavik Isstrom S,CR,1.03,W72.90N,-19.585277
10,Rimfaxe Gl.,CR,1.06,E63.35N,2.304114
15,Store Gl.,CR,1.29,W70.55N,10.092501
12,Kangiata Nunaata Se.,CR,1.50,W64.25N,-107.534485
14,Rink Isbrae,CR,1.78,W71.65N,-31.616169
2,Perlerfiup Se.,DW,0.70,W70.90N,-244.187242
0,F. Graae Gl.,DW,0.73,E71.75N,-62.792085
19,Kangerlussuaq Gl.,DW,1.10,E68.80N,-304.035157


In [130]:
# apply linear regresion using numpy
def linReg(x, y):
    '''linear regression using numpy starting from two one dimensional numpy arrays'''
    A = np.vstack([x, np.ones(len(x))]).T
    slope, intercept = np.linalg.lstsq(A, y, rcond=None)[0]
    return pd.Series({'slope':slope, 'intercept': intercept})

def lrr(row):
    """Linear regression on a row"""
    seconds0 = np.array(list(x.timestamp() for x in row.stab_time))
    ice0 = np.array(row.stab_ice) / (row.w21_mean_fjord_width * 1000.)
    
    seconds = seconds0 - seconds0[0]
    days = seconds / 86400.
    years = days / 365.
    ice = ice0 - ice0[0]    
    keep = (seconds > 30*86400)

    return linReg(years, ice)[['slope']]

def glacier_stability(df):
    """glacier_df: Data from a single glacier"""
    df = df.copy()   # Avoid warnings about setting on a copy of a slice
    df['advret_rate'] = df.apply(lrr, axis=1)
    df = df.drop(['stab_time', 'stab_ice'], axis=1)#[['year', 'sigma_max', 'advret_rate', 'w21_']]
    return df

def root_slope(df):
    """Takes output of glacier_stability()"""
    
    dfg = df.groupby(['ns642_GlacierID', 'year'])
    rows = list()
    for (glacier_id, year) ,dfx in dfg:
        row = {'glacier_id' : glacier_id, 'year' : year}
        row['poly_abc'] = abc = np.polyfit(df['sigma_max'], df['advret_rate'], 3)
        ab = np.polyder(abc)     # Derivative of abc
        rootsx = [np.real(x) for x in np.roots(abc) if ~np.iscomplex(x) and x >= 100000 and x <= 500000]
        if len(rootsx) > 0:
            row['root'] = root = rootsx[0]
            row['slope'] = np.poly1d(ab)(root)

    return pd.DataFrame(rows)


In [131]:
root_slope(df)

KeyError: 'advret_rate'

In [69]:
gs

year,2011,2012,2013,2014,2015,2016,2017,2018
sigma_max,,,,,,,,
100000.0,-5477.850009,-5332.383198,-5368.110325,-4757.492336,-4241.341942,-5281.606167,-5181.223874,-4881.505205
120000.0,-3237.463683,-3369.344727,-3695.747396,-2624.813255,-2294.062615,-2812.277661,-3605.254678,-2623.503908
140000.0,-2195.359873,-2139.242656,-2128.788747,-1897.961005,-1586.497526,-1814.261904,-2244.205872,-1932.263034
160000.0,-1433.046378,-1353.783920,-1458.023588,-934.572107,-978.454419,-1240.718345,-1870.558177,-1315.976456
180000.0,-1074.588501,-947.362735,-473.199018,-304.989689,-141.861488,-433.851952,-1362.328633,-558.962757
200000.0,-374.873931,-133.930475,194.868650,506.791777,477.999690,85.579171,-159.209526,362.138020
220000.0,-8.958016,646.853596,468.031506,617.066564,693.799880,179.722488,423.110675,775.562700
240000.0,101.206718,846.039638,782.451852,1222.535739,1290.039702,550.603920,777.911464,1155.068573
260000.0,339.948237,1469.604286,1182.889010,1298.536960,1556.651935,878.331842,1129.739284,1581.360530


In [24]:
import datetime
datetime.datetime(dt)

TypeError: an integer is required (got type real_datetime)

In [35]:
dt

real_datetime(2011, 1, 1, 0, 0)

In [27]:
dt.astype('datetime64[ns]')

AttributeError: 'real_datetime' object has no attribute 'astype'